In [1]:
import numpy as np

import pandas as pd

from gtda.time_series import SingleTakensEmbedding     # Implement the sliding window and the time delay embedding method.

from ripser import ripser                              # The Ripser package is used to compute Persistent Homology.

import matplotlib.pyplot as plt

import itertools                                       # This package is primarily used to handle iterable objects.

## （1）Sliding Window Method

In [3]:
# sliding shift = 1 

# signal is a 1-dimensional array of time series data.

def sliding_windows(signal, window_size):
    
    window = SingleTakensEmbedding(parameters_type = "fixed", dimension = window_size)
    
    sliding_windows = window.fit_transform(signal)
    
    return sliding_windows

## （2）TDA and Feature Extraction

### <font color = SteelBlue >Takens Embedding and Persistent Homology</font>

In [5]:
#                                                                                                                                   #
#  The takens_embedding method transforms a time series signal into a point cloud.                                                  #
#                                                                                                                                   #
#  The persistent_homology method is used to compute Persistent Homology and transforms a point cloudd into a persistence diagram.  #
#                                                                                                                                   #
#####################################################################################################################################

def takens_embedding(signal, embedding_dimension):
    
    embedder = SingleTakensEmbedding(parameters_type = "fixed", dimension = embedding_dimension)
    
    point_cloud = embedder.fit_transform(signal)
    
    return point_cloud

def persistent_homology(point_cloud):
    
    persistence_diagram = ripser(point_cloud)['dgms']
    
    return persistence_diagram 

### <font color = SteelBlue >Feature Extraction 1：get_diagram_lives</font>

In [8]:
# Compute the life(death - birth) of all points in the persistence diagram. 

def get_diagram_lives(diagram):
    
    return diagram[:,1] - diagram[:,0] if len(diagram) > 0 else np.array([0])

### <font color = SteelBlue >Feature Extraction 2：get_diagram_entropy</font>

In [10]:
# Compute the persistence entropy.

def get_diagram_entropy(lives):
    
    if max(abs(lives)) == 0:
        
        return 0.
        
    else:
        
        normalized_lives = lives / sum(lives)
        
        return sum(-normalized_lives * np.log(normalized_lives))

### <font color = SteelBlue >Feature Extraction 3：get_diagram_features</font>

In [12]:
# The get_diagram_features method organizes all three features into a single array.

def get_diagram_features(diagram):
    
    diagram = diagram[~np.any(np.isinf(diagram), axis = 1)]   # Remove points with inf values from the persistence diagram (for H0).
    
    lives = get_diagram_lives(diagram)
    
    bottleneck_distance = max(lives) / np.sqrt(2)        # feature 1（ bottleneck distance ）
    
    wasserstein_distance = sum(lives) / np.sqrt(2)       # feature 2（ wasserstein distance ）
    
    persistence_entropy = get_diagram_entropy(lives)     # feature 3（ persistence entropy ）
    
    result = [bottleneck_distance, wasserstein_distance, persistence_entropy] 
    
    return result

## （3）UCR Time Series Dataset

In [14]:
# This method splits the dataset into training and testing sets and converts them into DataFrames.

def beef_data():
    
    # Read a txt file and convert it into a DataFrame.
    
    data_train = pd.read_csv('../UCR_datasets/Beef_TRAIN.txt', sep = '\s+', header = None)
    
    data_test = pd.read_csv('../UCR_datasets/Beef_TEST.txt', sep = '\s+', header = None)
    
    # Separate the targets from the time series signal.
    
    train_targets = data_train.iloc[:, 0].astype(int)  
    
    train_signals = data_train.iloc[:, 1:].astype(float)  

    test_targets = data_test.iloc[:, 0].astype(int)  
    
    test_signals = data_test.iloc[:, 1:].astype(float)  

    # Convert the time series data in each row into a list format.
    
    train_signalss = []
    
    test_signalss = []
    
    train_signals_list = train_signals.values.tolist()
    
    for row in train_signals_list:
        
        train_signalss.append(row)
    
    test_signals_list = test_signals.values.tolist()
    
    for row in test_signals_list:
        
        test_signalss.append(row)

    # Create a new DataFrame where the signal is in list format.
    
    df_train = pd.DataFrame({
        
        'signal': train_signalss,
        
        'target': train_targets
        
    })

    df_test = pd.DataFrame({
        
        'signal': test_signalss,
        
        'target': test_targets
        
    })

    return df_train, df_test

In [15]:
# This method splits the dataset into training and testing sets and converts them into DataFrames.

def coffee_data():
    
    # Read a txt file and convert it into a DataFrame.
    
    data_train = pd.read_csv('../UCR_datasets/Coffee_TRAIN.txt', sep='\s+', header=None)
    
    data_test = pd.read_csv('../UCR_datasets/Coffee_TEST.txt', sep='\s+', header=None)
    
    # Separate the targets from the time series signal.
    
    train_targets = data_train.iloc[:, 0].astype(int)  
    
    train_signals = data_train.iloc[:, 1:].astype(float)  

    test_targets = data_test.iloc[:, 0].astype(int)  
    
    test_signals = data_test.iloc[:, 1:].astype(float)  

    # Convert the time series data in each row into a list format.
    
    train_signalss = []
    
    test_signalss = []
    
    train_signals_list = train_signals.values.tolist()
    
    for row in train_signals_list:
        
        train_signalss.append(row)
    
    test_signals_list = test_signals.values.tolist()
    
    for row in test_signals_list:
        
        test_signalss.append(row)

    # Create a new DataFrame where the signal is in list format.
    
    df_train = pd.DataFrame({
        
        'signal': train_signalss,
        
        'target': train_targets
        
    })

    df_test = pd.DataFrame({
        
        'signal': test_signalss,
        
        'target': test_targets
        
    })

    return df_train, df_test

In [16]:
# This method splits the dataset into training and testing sets and converts them into DataFrames.

def ham_data():
    
    # Read a txt file and convert it into a DataFrame.
    
    data_train = pd.read_csv('../UCR_datasets/Ham_TRAIN.txt', sep='\s+', header=None)
    
    data_test = pd.read_csv('../UCR_datasets/Ham_TEST.txt', sep='\s+', header=None)
    
    # Separate the targets from the time series signal.
    
    train_targets = data_train.iloc[:, 0].astype(int)  
    
    train_signals = data_train.iloc[:, 1:].astype(float)  

    test_targets = data_test.iloc[:, 0].astype(int)  
    
    test_signals = data_test.iloc[:, 1:].astype(float)  

    # Convert the time series data in each row into a list format.
    
    train_signalss = []
    
    test_signalss = []
    
    train_signals_list = train_signals.values.tolist()
    
    for row in train_signals_list:
        
        train_signalss.append(row)
    
    test_signals_list = test_signals.values.tolist()
    
    for row in test_signals_list:
        
        test_signalss.append(row)

    # Create a new DataFrame where the signal is in list format.
    
    df_train = pd.DataFrame({
        
        'signal': train_signalss,
        
        'target': train_targets
        
    })

    df_test = pd.DataFrame({
        
        'signal': test_signalss,
        
        'target': test_targets
        
    })

    return df_train, df_test

In [17]:
# This method splits the dataset into training and testing sets and converts them into DataFrames.

def meat_data():
    
    # Read a txt file and convert it into a DataFrame.
    
    data_train = pd.read_csv('../UCR_datasets/Meat_TRAIN.txt', sep='\s+', header=None)
    
    data_test = pd.read_csv('../UCR_datasets/Meat_TEST.txt', sep='\s+', header=None)
    
    # Separate the targets from the time series signal.
    
    train_targets = data_train.iloc[:, 0].astype(int)  
    
    train_signals = data_train.iloc[:, 1:].astype(float)  

    test_targets = data_test.iloc[:, 0].astype(int)  
    
    test_signals = data_test.iloc[:, 1:].astype(float)  

    # Convert the time series data in each row into a list format.
    
    train_signalss = []
    
    test_signalss = []
    
    train_signals_list = train_signals.values.tolist()
    
    for row in train_signals_list:
        
        train_signalss.append(row)
    
    test_signals_list = test_signals.values.tolist()
    
    for row in test_signals_list:
        
        test_signalss.append(row)

    # Create a new DataFrame where the signal is in list format.
    
    df_train = pd.DataFrame({
        
        'signal': train_signalss,
        
        'target': train_targets
        
    })

    df_test = pd.DataFrame({
        
        'signal': test_signalss,
        
        'target': test_targets
        
    })

    return df_train, df_test

In [18]:
# This method splits the dataset into training and testing sets and converts them into DataFrames.

def oliveoil_data():
    
    # Read a txt file and convert it into a DataFrame.
    
    data_train = pd.read_csv('../UCR_datasets/OliveOil_TRAIN.txt', sep='\s+', header=None)
    
    data_test = pd.read_csv('../UCR_datasets/OliveOil_TEST.txt', sep='\s+', header=None)
    
    # Separate the targets from the time series signal.
    
    train_targets = data_train.iloc[:, 0].astype(int)  
    
    train_signals = data_train.iloc[:, 1:].astype(float)  

    test_targets = data_test.iloc[:, 0].astype(int)  
    
    test_signals = data_test.iloc[:, 1:].astype(float)  

    # Convert the time series data in each row into a list format.
    
    train_signalss = []
    
    test_signalss = []
    
    train_signals_list = train_signals.values.tolist()
    
    for row in train_signals_list:
        
        train_signalss.append(row)
    
    test_signals_list = test_signals.values.tolist()
    
    for row in test_signals_list:
        
        test_signalss.append(row)

    # Create a new DataFrame where the signal is in list format.
    
    df_train = pd.DataFrame({
        
        'signal': train_signalss,
        
        'target': train_targets
        
    })

    df_test = pd.DataFrame({
        
        'signal': test_signalss,
        
        'target': test_targets
        
    })

    return df_train, df_test

In [19]:
# This method splits the dataset into training and testing sets and converts them into DataFrames.

def strawberry_data():
    
    # Read a txt file and convert it into a DataFrame.
    
    data_train = pd.read_csv('../UCR_datasets/Strawberry_TRAIN.txt', sep='\s+', header=None)
    
    data_test = pd.read_csv('../UCR_datasets/Strawberry_TEST.txt', sep='\s+', header=None)
    
    # Separate the targets from the time series signal.
    
    train_targets = data_train.iloc[:, 0].astype(int)  
    
    train_signals = data_train.iloc[:, 1:].astype(float)  

    test_targets = data_test.iloc[:, 0].astype(int)  
    
    test_signals = data_test.iloc[:, 1:].astype(float)  

    # Convert the time series data in each row into a list format.
    
    train_signalss = []
    
    test_signalss = []
    
    train_signals_list = train_signals.values.tolist()
    
    for row in train_signals_list:
        
        train_signalss.append(row)
    
    test_signals_list = test_signals.values.tolist()
    
    for row in test_signals_list:
        
        test_signalss.append(row)

    # Create a new DataFrame where the signal is in list format.
    
    df_train = pd.DataFrame({
        
        'signal': train_signalss,
        
        'target': train_targets
        
    })

    df_test = pd.DataFrame({
        
        'signal': test_signalss,
        
        'target': test_targets
        
    })

    return df_train, df_test

In [20]:
# This method splits the dataset into training and testing sets and converts them into DataFrames.

def wine_data():
    
    # Read a txt file and convert it into a DataFrame.
    
    data_train = pd.read_csv('../UCR_datasets/Wine_TRAIN.txt', sep='\s+', header=None)
    
    data_test = pd.read_csv('../UCR_datasets/Wine_TEST.txt', sep='\s+', header=None)
    
    # Separate the targets from the time series signal.
    
    train_targets = data_train.iloc[:, 0].astype(int)  
    
    train_signals = data_train.iloc[:, 1:].astype(float)  

    test_targets = data_test.iloc[:, 0].astype(int)  
    
    test_signals = data_test.iloc[:, 1:].astype(float)  

    # Convert the time series data in each row into a list format.
    
    train_signalss = []
    
    test_signalss = []
    
    train_signals_list = train_signals.values.tolist()
    
    for row in train_signals_list:
        
        train_signalss.append(row)
    
    test_signals_list = test_signals.values.tolist()
    
    for row in test_signals_list:
        
        test_signalss.append(row)

    # Create a new DataFrame where the signal is in list format.
    
    df_train = pd.DataFrame({
        
        'signal': train_signalss,
        
        'target': train_targets
        
    })

    df_test = pd.DataFrame({
        
        'signal': test_signalss,
        
        'target': test_targets
        
    })

    return df_train, df_test

## （4）Machine Learning 

In [22]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

In [23]:
# For Beef

def beef_classification_model(train_features, train_target, test_features, test_target):
    
    X_train = train_features
    
    y_train = train_target
    
    X_test = test_features
    
    y_test = test_target

    pipe = Pipeline([('scaler', StandardScaler()), ('bc', BaggingClassifier(estimator = LogisticRegression(max_iter = 3000, C = 0.01)))])
    
    param_grid = {
        
    'bc__n_estimators': [50],
        
    'bc__random_state': [14]
        
    }

    grid_search = GridSearchCV(pipe, param_grid, cv = 5, scoring = 'accuracy', verbose = 1, n_jobs = -1)
    
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    
    print(f"Best cross-validation accuracy: {grid_search.best_score_}")
    
    y_pred = grid_search.predict(X_test)

    bc_accuracy = round(100 * (accuracy_score(y_test, y_pred)), 3)

    bc_macro_f1 = round(100 * (f1_score(y_test, y_pred, average="macro")), 3)
    
    print('bc Test accuracy:', accuracy_score(y_test, y_pred))

    print("bc Test Macro-F1:", f1_score(y_test, y_pred, average="macro"))
    
    return bc_accuracy, bc_macro_f1

# For Coffee

def coffee_classification_model(train_features, train_target, test_features, test_target):
    
    X_train = train_features
    
    y_train = train_target
    
    X_test = test_features
    
    y_test = test_target

    pipe = Pipeline([('scaler', StandardScaler()), ('bc', BaggingClassifier(estimator = LogisticRegression(max_iter = 1800)))])
    
    param_grid = {
        
    'bc__n_estimators': [100, 200],
        
    'bc__max_samples': [1.0, 0.8, 0.6],
        
    'bc__random_state': [14]
        
    }

    grid_search = GridSearchCV(pipe, param_grid, cv = 3, scoring = 'accuracy', verbose = 1, n_jobs = -1)
    
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    
    print(f"Best cross-validation accuracy: {grid_search.best_score_}")
    
    y_pred = grid_search.predict(X_test)

    bc_accuracy = round(100 * (accuracy_score(y_test, y_pred)), 3)

    bc_macro_f1 = round(100 * (f1_score(y_test, y_pred, average="macro")), 3)
    
    print('bc Test accuracy:', accuracy_score(y_test, y_pred))

    print("bc Test Macro-F1:", f1_score(y_test, y_pred, average="macro"))
    
    return bc_accuracy, bc_macro_f1

# For Ham

def ham_classification_model(train_features, train_target, test_features, test_target):
    
    X_train = train_features
    
    y_train = train_target
    
    X_test = test_features
    
    y_test = test_target

    pipe = Pipeline([('scaler', StandardScaler()), ('bc', BaggingClassifier(estimator = LogisticRegression(max_iter = 2000, solver = 'lbfgs')))])
    
    param_grid = {
        
    'bc__n_estimators': [10, 20, 30, 50, 100],
        
    'bc__max_samples': [1.0, 0.8, 0.6],
        
    'bc__max_features': [1.0, 0.8],
        
    'bc__random_state': [45]
        
    }

    grid_search = GridSearchCV(pipe, param_grid, cv = 3, scoring = 'accuracy', verbose = 1, n_jobs = -1)
    
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    
    print(f"Best cross-validation accuracy: {grid_search.best_score_}")
    
    y_pred = grid_search.predict(X_test)

    bc_accuracy = round(100 * (accuracy_score(y_test, y_pred)), 3)

    bc_macro_f1 = round(100 * (f1_score(y_test, y_pred, average="macro")), 3)
    
    print('bc Test accuracy:', accuracy_score(y_test, y_pred))

    print("bc Test Macro-F1:", f1_score(y_test, y_pred, average="macro"))
    
    return bc_accuracy, bc_macro_f1

# For Meat

def meat_classification_model(train_features, train_target, test_features, test_target):
    
    X_train = train_features
    
    y_train = train_target
    
    X_test = test_features
    
    y_test = test_target

    pipe = Pipeline([('scaler', StandardScaler()), ('bc', BaggingClassifier(estimator = LogisticRegression(max_iter = 2000, C = 0.01, solver = 'sag')))])
    
    param_grid = {
        
    'bc__n_estimators': [10, 20, 30, 50, 100],
        
    'bc__max_samples': [1.0, 0.8, 0.6],
        
    'bc__max_features': [1.0, 0.8],
        
    'bc__random_state': [42]
        
    }

    grid_search = GridSearchCV(pipe, param_grid, cv = 3, scoring = 'accuracy', verbose = 1, n_jobs = -1)
    
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    
    print(f"Best cross-validation accuracy: {grid_search.best_score_}")
    
    y_pred = grid_search.predict(X_test)

    bc_accuracy = round(100 * (accuracy_score(y_test, y_pred)), 3)

    bc_macro_f1 = round(100 * (f1_score(y_test, y_pred, average="macro")), 3)
    
    print('bc Test accuracy:', accuracy_score(y_test, y_pred))

    print("bc Test Macro-F1:", f1_score(y_test, y_pred, average="macro"))
    
    return bc_accuracy, bc_macro_f1

# For Olive oil

def oliveoil_classification_model(train_features, train_target, test_features, test_target):
    
    X_train = train_features
    
    y_train = train_target
    
    X_test = test_features
    
    y_test = test_target

    pipe = Pipeline([('scaler', StandardScaler()), ('bc', BaggingClassifier(estimator = LogisticRegression(max_iter = 3500, C = 0.01, solver = 'sag')))])
    
    param_grid = {
        
    'bc__n_estimators': [10, 20, 30, 50, 100],
        
    'bc__max_samples': [1.0, 0.8, 0.6],
        
    'bc__max_features': [1.0, 0.8],
        
    'bc__random_state': [14]
        
    }

    grid_search = GridSearchCV(pipe, param_grid, cv = 3, scoring = 'accuracy', verbose = 1, n_jobs = -1)
    
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    
    print(f"Best cross-validation accuracy: {grid_search.best_score_}")
    
    y_pred = grid_search.predict(X_test)

    bc_accuracy = round(100 * (accuracy_score(y_test, y_pred)), 3)

    bc_macro_f1 = round(100 * (f1_score(y_test, y_pred, average="macro")), 3)
    
    print('bc Test accuracy:', accuracy_score(y_test, y_pred))

    print("bc Test Macro-F1:", f1_score(y_test, y_pred, average="macro"))
    
    return bc_accuracy, bc_macro_f1
    
# For Strawberry 

def strawberry_classification_model(train_features, train_target, test_features, test_target):
    
    X_train = train_features
    
    y_train = train_target
    
    X_test = test_features
    
    y_test = test_target

    pipe = Pipeline([('scaler', StandardScaler()), ('bc', BaggingClassifier(estimator = KNeighborsClassifier(n_neighbors = 1, algorithm = 'ball_tree', leaf_size = 20, p = 1)))])
    
    param_grid = {
        
    'bc__n_estimators': [10, 20, 30, 50, 100],
        
    'bc__max_samples': [1.0, 0.8, 0.6],
        
    'bc__max_features': [1.0, 0.8],
        
    'bc__random_state': [14]
        
    }

    grid_search = GridSearchCV(pipe, param_grid, cv = 3, scoring = 'accuracy', verbose = 1, n_jobs = -1)
    
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    
    print(f"Best cross-validation accuracy: {grid_search.best_score_}")
    
    y_pred = grid_search.predict(X_test)

    bc_accuracy = round(100 * (accuracy_score(y_test, y_pred)), 3)

    bc_macro_f1 = round(100 * (f1_score(y_test, y_pred, average="macro")), 3)
    
    print('bc Test accuracy:', accuracy_score(y_test, y_pred))

    print("bc Test Macro-F1:", f1_score(y_test, y_pred, average="macro"))
    
    return bc_accuracy, bc_macro_f1

# For Wine

def wine_classification_model(train_features, train_target, test_features, test_target):
    
    X_train = train_features
    
    y_train = train_target
    
    X_test = test_features
    
    y_test = test_target

    pipe = Pipeline([('scaler', StandardScaler()), ('bc', BaggingClassifier(estimator = LogisticRegression(max_iter = 1800)))])
    
    param_grid = {
        
    'bc__n_estimators': [10, 30, 50, 100],
        
    'bc__max_samples': [1.0, 0.8, 0.6],
        
    'bc__random_state': [3]
        
    }

    grid_search = GridSearchCV(pipe, param_grid, cv = 3, scoring = 'accuracy', verbose = 1, n_jobs = -1)
    
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    
    print(f"Best cross-validation accuracy: {grid_search.best_score_}")
    
    y_pred = grid_search.predict(X_test)

    bc_accuracy = round(100 * (accuracy_score(y_test, y_pred)), 3)

    bc_macro_f1 = round(100 * (f1_score(y_test, y_pred, average="macro")), 3)
    
    print('bc Test accuracy:', accuracy_score(y_test, y_pred))

    print("bc Test Macro-F1:", f1_score(y_test, y_pred, average="macro"))
    
    return bc_accuracy, bc_macro_f1


## （5）Convert Series to DataFrame

In [25]:
# convert pd.Series([[1,2,3],[4,5,6]]) to pd.DataFrame([[1,2,3],[4,5,6]])

def pd_series_to_dataframe(series):
    
    return pd.DataFrame.from_dict(dict(zip(series.index, series.values))).T

## （6）Main Experiment

In [27]:
# beef main experiment

def beef_experiment(embedding_dimension = 2):

    all_classification_accuracy = []   # Store all accuracy results.
    all_classification_macro_f1 = []   # Store all macro-f1 results.
    
    # Step 1. Call the beef_data() method to obtain the beef dataset and convert it into Series format.
    
    train_data, test_data = beef_data()

    train_data_series = pd.Series(train_data['signal'].iloc[:])
    
    test_data_series = pd.Series(test_data['signal'].iloc[:])

    window_size = [5, 10, 45, 50, 95, 100]        # (5, 10) = small size，(45, 50) = medium size，(95, 100) = large size

    # Step 2. Apply a sliding window method to all time series in the dataset.
    
    for size in window_size:
        
        train_sliding_windows = train_data_series.map(lambda x: sliding_windows(x, size))
        
        test_sliding_windows = test_data_series.map(lambda x: sliding_windows(x, size))

        # Step 3. Apply time-delay embedding method to each window and obtain a 2-dimensional point cloud.
        
        train_point_cloud = train_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])
        
        test_point_cloud = test_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])

        # Step 4. Convert each point cloud into a persistence diagram.
        
        train_persistence_diagrams = train_point_cloud.map(lambda x: [persistent_homology(row) for row in x])
        
        test_persistence_diagrams = test_point_cloud.map(lambda x: [persistent_homology(row) for row in x])

        # Step 5. Separate the H0 and H1 structures in the persistence diagram.
        
        train_H0 = train_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        train_H1 = train_persistence_diagrams.map(lambda x: [row[1] for row in x])

        test_H0 = test_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        test_H1 = test_persistence_diagrams.map(lambda x: [row[1] for row in x])

        # Step 6. Feature Extraction !

        features_train_H0 = train_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_train_H1 = train_H1.map(lambda x: [get_diagram_features(row) for row in x])

        features_test_H0 = test_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_test_H1 = test_H1.map(lambda x: [get_diagram_features(row) for row in x])

        # Step 7. Use the itertools package to merge the features of each window into a 1-dimensional array.
        
        features_train_H0 = features_train_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_train_H1 = features_train_H1.map(lambda x: list(itertools.chain(*x)))

        features_test_H0 = features_test_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_test_H1 = features_test_H1.map(lambda x: list(itertools.chain(*x)))

        # Step 8. Convert all features of H0 and H1 into a DataFrame format, and merge them.
        
        features_train_H0 = pd_series_to_dataframe(features_train_H0)
        
        features_train_H1 = pd_series_to_dataframe(features_train_H1)

        features_test_H0 = pd_series_to_dataframe(features_test_H0)
        
        features_test_H1 = pd_series_to_dataframe(features_test_H1)

        features_train = pd.concat([features_train_H0, features_train_H1], axis=1)   
        
        features_train.columns = np.arange(features_train.shape[1])
        
        features_test = pd.concat([features_test_H0, features_test_H1], axis=1)
        
        features_test.columns = np.arange(features_test.shape[1])

        # Step 9. Convert all features in each row into a list format.
        
        train_features = []
        
        features_train = features_train.values.tolist()
        
        for row in features_train:
            
            train_features.append(row)

        test_features = []
        
        features_test = features_test.values.tolist()
        
        for row in features_test:
            
            test_features.append(row)

        # Step 10. Combine all features into the original DataFrame.
        
        train_data['features'] = train_features
        
        test_data['features'] = test_features

        # Step 11. Machine Learning
        
        train_features = np.vstack(train_data['features'].values)
        
        train_target = train_data['target'].values
        
        test_features = np.vstack(test_data['features'].values)
        
        test_target = test_data['target'].values

        bc_accuracy, bc_macro_f1 = beef_classification_model(train_features, train_target, test_features, test_target)

        # Step 12. Organize the classification results into a DataFrame.
        
        classification_accuracy = [bc_accuracy]

        classification_macor_f1 = [bc_macro_f1]
        
        all_classification_accuracy.append(classification_accuracy)

        all_classification_macro_f1.append(classification_macor_f1)
        
        print('window size ', size,' execution completed !')

        print()

    all_classification_accuracy = np.array(all_classification_accuracy)                                      
    
    all_classification_accuracy = pd.DataFrame(all_classification_accuracy, columns = ['Accuracy'])   
    
    all_classification_accuracy.index = window_size

    all_classification_macro_f1 = np.array(all_classification_macro_f1)                                      
    
    all_classification_macro_f1 = pd.DataFrame(all_classification_macro_f1, columns = ['Macro-F1'])   
    
    all_classification_macro_f1.index = window_size
 
    return all_classification_accuracy, all_classification_macro_f1


In [28]:
# coffee main experiment

def coffee_experiment(embedding_dimension = 2):

    all_classification_accuracy = []   # Store all accuracy results.
    all_classification_macro_f1 = []   # Store all macro-f1 results.
    
    # Step 1. Call the coffee_data() method to obtain the beef dataset and convert it into Series format.
    
    train_data, test_data = coffee_data()

    train_data_series = pd.Series(train_data['signal'].iloc[:])
    
    test_data_series = pd.Series(test_data['signal'].iloc[:])

    window_size = [5, 10, 45, 50, 95, 100]        # (5, 10) = small size，(45, 50) = medium size，(95, 100) = large size

    # Step 2. Apply a sliding window method to all time series in the dataset.
    
    for size in window_size:
        
        train_sliding_windows = train_data_series.map(lambda x: sliding_windows(x, size))
        
        test_sliding_windows = test_data_series.map(lambda x: sliding_windows(x, size))

        # Step 3. Apply time-delay embedding method to each window and obtain a 2-dimensional point cloud.
        
        train_point_cloud = train_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])
        
        test_point_cloud = test_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])

        # Step 4. Convert each point cloud into a persistence diagram.
        
        train_persistence_diagrams = train_point_cloud.map(lambda x: [persistent_homology(row) for row in x])
        
        test_persistence_diagrams = test_point_cloud.map(lambda x: [persistent_homology(row) for row in x])

        # Step 5. Separate the H0 and H1 structures in the persistence diagram.
        
        train_H0 = train_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        train_H1 = train_persistence_diagrams.map(lambda x: [row[1] for row in x])

        test_H0 = test_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        test_H1 = test_persistence_diagrams.map(lambda x: [row[1] for row in x])

        # Step 6. Feature Extraction !

        features_train_H0 = train_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_train_H1 = train_H1.map(lambda x: [get_diagram_features(row) for row in x])

        features_test_H0 = test_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_test_H1 = test_H1.map(lambda x: [get_diagram_features(row) for row in x])

        # Step 7. Use the itertools package to merge the features of each window into a 1-dimensional array.
        
        features_train_H0 = features_train_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_train_H1 = features_train_H1.map(lambda x: list(itertools.chain(*x)))

        features_test_H0 = features_test_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_test_H1 = features_test_H1.map(lambda x: list(itertools.chain(*x)))

        # Step 8. Convert all features of H0 and H1 into a DataFrame format, and merge them.
        
        features_train_H0 = pd_series_to_dataframe(features_train_H0)
        
        features_train_H1 = pd_series_to_dataframe(features_train_H1)

        features_test_H0 = pd_series_to_dataframe(features_test_H0)
        
        features_test_H1 = pd_series_to_dataframe(features_test_H1)

        features_train = pd.concat([features_train_H0, features_train_H1], axis=1)   
        
        features_train.columns = np.arange(features_train.shape[1])
        
        features_test = pd.concat([features_test_H0, features_test_H1], axis=1)
        
        features_test.columns = np.arange(features_test.shape[1])

        # Step 9. Convert all features in each row into a list format.
        
        train_features = []
        
        features_train = features_train.values.tolist()
        
        for row in features_train:
            
            train_features.append(row)

        test_features = []
        
        features_test = features_test.values.tolist()
        
        for row in features_test:
            
            test_features.append(row)

        # Step 10. Combine all features into the original DataFrame.
        
        train_data['features'] = train_features
        
        test_data['features'] = test_features

        # Step 11. Machine Learning
        
        train_features = np.vstack(train_data['features'].values)
        
        train_target = train_data['target'].values
        
        test_features = np.vstack(test_data['features'].values)
        
        test_target = test_data['target'].values

        bc_accuracy, bc_macro_f1 = coffee_classification_model(train_features, train_target, test_features, test_target)

        # Step 12. Organize the classification results into a DataFrame.
        
        classification_accuracy = [bc_accuracy]

        classification_macor_f1 = [bc_macro_f1]
        
        all_classification_accuracy.append(classification_accuracy)

        all_classification_macro_f1.append(classification_macor_f1)
        
        print('window size ', size,' execution completed !')

        print()

    all_classification_accuracy = np.array(all_classification_accuracy)                                      
    
    all_classification_accuracy = pd.DataFrame(all_classification_accuracy, columns = ['Accuracy'])   
    
    all_classification_accuracy.index = window_size

    all_classification_macro_f1 = np.array(all_classification_macro_f1)                                      
    
    all_classification_macro_f1 = pd.DataFrame(all_classification_macro_f1, columns = ['Macro-F1'])   
    
    all_classification_macro_f1.index = window_size
 
    return all_classification_accuracy, all_classification_macro_f1


In [29]:
# ham main experiment

def ham_experiment(embedding_dimension = 2):

    all_classification_accuracy = []   # Store all accuracy results.
    all_classification_macro_f1 = []   # Store all macro-f1 results.
    
    # Step 1. Call the ham_data() method to obtain the beef dataset and convert it into Series format.
    
    train_data, test_data = ham_data()

    train_data_series = pd.Series(train_data['signal'].iloc[:])
    
    test_data_series = pd.Series(test_data['signal'].iloc[:])

    window_size = [5, 10, 45, 50, 95, 100]        # (5, 10) = small size，(45, 50) = medium size，(95, 100) = large size

    # Step 2. Apply a sliding window method to all time series in the dataset.
    
    for size in window_size:
        
        train_sliding_windows = train_data_series.map(lambda x: sliding_windows(x, size))
        
        test_sliding_windows = test_data_series.map(lambda x: sliding_windows(x, size))

        # Step 3. Apply time-delay embedding method to each window and obtain a 2-dimensional point cloud.
        
        train_point_cloud = train_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])
        
        test_point_cloud = test_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])

        # Step 4. Convert each point cloud into a persistence diagram.
        
        train_persistence_diagrams = train_point_cloud.map(lambda x: [persistent_homology(row) for row in x])
        
        test_persistence_diagrams = test_point_cloud.map(lambda x: [persistent_homology(row) for row in x])

        # Step 5. Separate the H0 and H1 structures in the persistence diagram.
        
        train_H0 = train_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        train_H1 = train_persistence_diagrams.map(lambda x: [row[1] for row in x])

        test_H0 = test_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        test_H1 = test_persistence_diagrams.map(lambda x: [row[1] for row in x])

        # Step 6. Feature Extraction !

        features_train_H0 = train_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_train_H1 = train_H1.map(lambda x: [get_diagram_features(row) for row in x])

        features_test_H0 = test_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_test_H1 = test_H1.map(lambda x: [get_diagram_features(row) for row in x])

        # Step 7. Use the itertools package to merge the features of each window into a 1-dimensional array.
        
        features_train_H0 = features_train_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_train_H1 = features_train_H1.map(lambda x: list(itertools.chain(*x)))

        features_test_H0 = features_test_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_test_H1 = features_test_H1.map(lambda x: list(itertools.chain(*x)))

        # Step 8. Convert all features of H0 and H1 into a DataFrame format, and merge them.
        
        features_train_H0 = pd_series_to_dataframe(features_train_H0)
        
        features_train_H1 = pd_series_to_dataframe(features_train_H1)

        features_test_H0 = pd_series_to_dataframe(features_test_H0)
        
        features_test_H1 = pd_series_to_dataframe(features_test_H1)

        features_train = pd.concat([features_train_H0, features_train_H1], axis=1)  
        
        features_train.columns = np.arange(features_train.shape[1])
        
        features_test = pd.concat([features_test_H0, features_test_H1], axis=1)
        
        features_test.columns = np.arange(features_test.shape[1])

        # Step 9. Convert all features in each row into a list format.
        
        train_features = []
        
        features_train = features_train.values.tolist()
        
        for row in features_train:
            
            train_features.append(row)

        test_features = []
        
        features_test = features_test.values.tolist()
        
        for row in features_test:
            
            test_features.append(row)

        # Step 10. Combine all features into the original DataFrame.
        
        train_data['features'] = train_features
        
        test_data['features'] = test_features

        # Step 11. Machine Learning
        
        train_features = np.vstack(train_data['features'].values)
        
        train_target = train_data['target'].values
        
        test_features = np.vstack(test_data['features'].values)
        
        test_target = test_data['target'].values

        bc_accuracy, bc_macro_f1 = ham_classification_model(train_features, train_target, test_features, test_target)

        # Step 12. Organize the classification results into a DataFrame.
        
        classification_accuracy = [bc_accuracy]

        classification_macor_f1 = [bc_macro_f1]
        
        all_classification_accuracy.append(classification_accuracy)

        all_classification_macro_f1.append(classification_macor_f1)
        
        print('window size ', size,' execution completed !')

        print()

    all_classification_accuracy = np.array(all_classification_accuracy)                                      
    
    all_classification_accuracy = pd.DataFrame(all_classification_accuracy, columns = ['Accuracy'])   
    
    all_classification_accuracy.index = window_size

    all_classification_macro_f1 = np.array(all_classification_macro_f1)                                      
    
    all_classification_macro_f1 = pd.DataFrame(all_classification_macro_f1, columns = ['Macro-F1'])   
    
    all_classification_macro_f1.index = window_size
 
    return all_classification_accuracy, all_classification_macro_f1


In [30]:
# meat main experiment

def meat_experiment(embedding_dimension = 2):

    all_classification_accuracy = []   # Store all accuracy results.
    all_classification_macro_f1 = []   # Store all macro-f1 results.
    
    # Step 1. Call the meat_data() method to obtain the beef dataset and convert it into Series format.
    
    train_data, test_data = meat_data()

    train_data_series = pd.Series(train_data['signal'].iloc[:])
    
    test_data_series = pd.Series(test_data['signal'].iloc[:])

    window_size = [5, 10, 45, 50, 95, 100]        # (5, 10) = small size，(45, 50) = medium size，(95, 100) = large size

    # Step 2. Apply a sliding window method to all time series in the dataset.
    
    for size in window_size:
        
        train_sliding_windows = train_data_series.map(lambda x: sliding_windows(x, size))
        
        test_sliding_windows = test_data_series.map(lambda x: sliding_windows(x, size))

        # Step 3. Apply time-delay embedding method to each window and obtain a 2-dimensional point cloud.
        
        train_point_cloud = train_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])
        
        test_point_cloud = test_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])

        # Step 4. Convert each point cloud into a persistence diagram.
        
        train_persistence_diagrams = train_point_cloud.map(lambda x: [persistent_homology(row) for row in x])
        
        test_persistence_diagrams = test_point_cloud.map(lambda x: [persistent_homology(row) for row in x])

        # Step 5. Separate the H0 and H1 structures in the persistence diagram.
        
        train_H0 = train_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        train_H1 = train_persistence_diagrams.map(lambda x: [row[1] for row in x])

        test_H0 = test_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        test_H1 = test_persistence_diagrams.map(lambda x: [row[1] for row in x])

        # Step 6. Feature Extraction !

        features_train_H0 = train_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_train_H1 = train_H1.map(lambda x: [get_diagram_features(row) for row in x])

        features_test_H0 = test_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_test_H1 = test_H1.map(lambda x: [get_diagram_features(row) for row in x])

        # Step 7. Use the itertools package to merge the features of each window into a 1-dimensional array.
        
        features_train_H0 = features_train_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_train_H1 = features_train_H1.map(lambda x: list(itertools.chain(*x)))

        features_test_H0 = features_test_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_test_H1 = features_test_H1.map(lambda x: list(itertools.chain(*x)))

        # Step 8. Convert all features of H0 and H1 into a DataFrame format, and merge them.
        
        features_train_H0 = pd_series_to_dataframe(features_train_H0)
        
        features_train_H1 = pd_series_to_dataframe(features_train_H1)

        features_test_H0 = pd_series_to_dataframe(features_test_H0)
        
        features_test_H1 = pd_series_to_dataframe(features_test_H1)

        features_train = pd.concat([features_train_H0, features_train_H1], axis=1)   
        
        features_train.columns = np.arange(features_train.shape[1])
        
        features_test = pd.concat([features_test_H0, features_test_H1], axis=1)
        
        features_test.columns = np.arange(features_test.shape[1])

        # Step 9. Convert all features in each row into a list format.
        
        train_features = []
        
        features_train = features_train.values.tolist()
        
        for row in features_train:
            
            train_features.append(row)

        test_features = []
        
        features_test = features_test.values.tolist()
        
        for row in features_test:
            
            test_features.append(row)

        # Step 10. Combine all features into the original DataFrame.
        
        train_data['features'] = train_features
        
        test_data['features'] = test_features

        # Step 11. Machine Learning
        
        train_features = np.vstack(train_data['features'].values)
        
        train_target = train_data['target'].values
        
        test_features = np.vstack(test_data['features'].values)
        
        test_target = test_data['target'].values

        bc_accuracy, bc_macro_f1 = meat_classification_model(train_features, train_target, test_features, test_target)

        # Step 12. Organize the classification results into a DataFrame.
        
        classification_accuracy = [bc_accuracy]

        classification_macor_f1 = [bc_macro_f1]
        
        all_classification_accuracy.append(classification_accuracy)

        all_classification_macro_f1.append(classification_macor_f1)
        
        print('window size ', size,' execution completed !')

        print()

    all_classification_accuracy = np.array(all_classification_accuracy)                                      
    
    all_classification_accuracy = pd.DataFrame(all_classification_accuracy, columns = ['Accuracy'])   
    
    all_classification_accuracy.index = window_size

    all_classification_macro_f1 = np.array(all_classification_macro_f1)                                      
    
    all_classification_macro_f1 = pd.DataFrame(all_classification_macro_f1, columns = ['Macro-F1'])   
    
    all_classification_macro_f1.index = window_size
 
    return all_classification_accuracy, all_classification_macro_f1


In [31]:
# olive oil main experiment

def oliveoil_experiment(embedding_dimension = 2):

    all_classification_accuracy = []   # Store all accuracy results.
    all_classification_macro_f1 = []   # Store all macro-f1 results.
    
    # Step 1. Call the oliveoil_data() method to obtain the beef dataset and convert it into Series format.
    
    train_data, test_data = oliveoil_data()

    train_data_series = pd.Series(train_data['signal'].iloc[:])
    
    test_data_series = pd.Series(test_data['signal'].iloc[:])

    window_size = [5, 10, 45, 50, 95, 100]        # (5, 10) = small size，(45, 50) = medium size，(95, 100) = large size

    # Step 2. Apply a sliding window method to all time series in the dataset.
    
    for size in window_size:
        
        train_sliding_windows = train_data_series.map(lambda x: sliding_windows(x, size))
        
        test_sliding_windows = test_data_series.map(lambda x: sliding_windows(x, size))

        # Step 3. Apply time-delay embedding method to each window and obtain a 2-dimensional point cloud.
        
        train_point_cloud = train_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])
        
        test_point_cloud = test_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])

        # Step 4. Convert each point cloud into a persistence diagram.
        
        train_persistence_diagrams = train_point_cloud.map(lambda x: [persistent_homology(row) for row in x])
        
        test_persistence_diagrams = test_point_cloud.map(lambda x: [persistent_homology(row) for row in x])

        # Step 5. Separate the H0 and H1 structures in the persistence diagram.
        
        train_H0 = train_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        train_H1 = train_persistence_diagrams.map(lambda x: [row[1] for row in x])

        test_H0 = test_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        test_H1 = test_persistence_diagrams.map(lambda x: [row[1] for row in x])

        # Step 6. Feature Extraction !

        features_train_H0 = train_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_train_H1 = train_H1.map(lambda x: [get_diagram_features(row) for row in x])

        features_test_H0 = test_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_test_H1 = test_H1.map(lambda x: [get_diagram_features(row) for row in x])

        # Step 7. Use the itertools package to merge the features of each window into a 1-dimensional array.
        
        features_train_H0 = features_train_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_train_H1 = features_train_H1.map(lambda x: list(itertools.chain(*x)))

        features_test_H0 = features_test_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_test_H1 = features_test_H1.map(lambda x: list(itertools.chain(*x)))

        # Step 8. Convert all features of H0 and H1 into a DataFrame format, and merge them.
        
        features_train_H0 = pd_series_to_dataframe(features_train_H0)
        
        features_train_H1 = pd_series_to_dataframe(features_train_H1)

        features_test_H0 = pd_series_to_dataframe(features_test_H0)
        
        features_test_H1 = pd_series_to_dataframe(features_test_H1)

        features_train = pd.concat([features_train_H0, features_train_H1], axis=1)   
        
        features_train.columns = np.arange(features_train.shape[1])
        
        features_test = pd.concat([features_test_H0, features_test_H1], axis=1)
        
        features_test.columns = np.arange(features_test.shape[1])

        # Step 9. Convert all features in each row into a list format.
        
        train_features = []
        
        features_train = features_train.values.tolist()
        
        for row in features_train:
            
            train_features.append(row)

        test_features = []
        
        features_test = features_test.values.tolist()
        
        for row in features_test:
            
            test_features.append(row)

        # Step 10. Combine all features into the original DataFrame.
        
        train_data['features'] = train_features
        
        test_data['features'] = test_features

        # Step 11. Machine Learning
        
        train_features = np.vstack(train_data['features'].values)
        
        train_target = train_data['target'].values
        
        test_features = np.vstack(test_data['features'].values)
        
        test_target = test_data['target'].values

        bc_accuracy, bc_macro_f1 = oliveoil_classification_model(train_features, train_target, test_features, test_target)

        # Step 12. Organize the classification results into a DataFrame.
        
        classification_accuracy = [bc_accuracy]

        classification_macor_f1 = [bc_macro_f1]
        
        all_classification_accuracy.append(classification_accuracy)

        all_classification_macro_f1.append(classification_macor_f1)
        
        print('window size ', size,' execution completed !')

        print()

    all_classification_accuracy = np.array(all_classification_accuracy)                                      
    
    all_classification_accuracy = pd.DataFrame(all_classification_accuracy, columns = ['Accuracy'])   
    
    all_classification_accuracy.index = window_size

    all_classification_macro_f1 = np.array(all_classification_macro_f1)                                      
    
    all_classification_macro_f1 = pd.DataFrame(all_classification_macro_f1, columns = ['Macro-F1'])   
    
    all_classification_macro_f1.index = window_size
 
    return all_classification_accuracy, all_classification_macro_f1


In [32]:
# strawberry main experiment

def strawberry_experiment(embedding_dimension = 2):

    all_classification_accuracy = []   # Store all accuracy results.
    all_classification_macro_f1 = []   # Store all macro-f1 results.
    
    # Step 1. Call the strawberry_data() method to obtain the beef dataset and convert it into Series format.
    
    train_data, test_data = strawberry_data()

    train_data_series = pd.Series(train_data['signal'].iloc[:])
    
    test_data_series = pd.Series(test_data['signal'].iloc[:])

    window_size = [5, 10, 45, 50, 95, 100]        # (5, 10) = small size，(45, 50) = medium size，(95, 100) = large size

    # Step 2. Apply a sliding window method to all time series in the dataset.
    
    for size in window_size:
        
        train_sliding_windows = train_data_series.map(lambda x: sliding_windows(x, size))
        
        test_sliding_windows = test_data_series.map(lambda x: sliding_windows(x, size))

        # Step 3. Apply time-delay embedding method to each window and obtain a 2-dimensional point cloud.
        
        train_point_cloud = train_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])
        
        test_point_cloud = test_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])

        # Step 4. Convert each point cloud into a persistence diagram.
        
        train_persistence_diagrams = train_point_cloud.map(lambda x: [persistent_homology(row) for row in x])
        
        test_persistence_diagrams = test_point_cloud.map(lambda x: [persistent_homology(row) for row in x])

        # Step 5. Separate the H0 and H1 structures in the persistence diagram.
        
        train_H0 = train_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        train_H1 = train_persistence_diagrams.map(lambda x: [row[1] for row in x])

        test_H0 = test_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        test_H1 = test_persistence_diagrams.map(lambda x: [row[1] for row in x])

        # Step 6. Feature Extraction !

        features_train_H0 = train_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_train_H1 = train_H1.map(lambda x: [get_diagram_features(row) for row in x])

        features_test_H0 = test_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_test_H1 = test_H1.map(lambda x: [get_diagram_features(row) for row in x])

        # Step 7. Use the itertools package to merge the features of each window into a 1-dimensional array.
        
        features_train_H0 = features_train_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_train_H1 = features_train_H1.map(lambda x: list(itertools.chain(*x)))

        features_test_H0 = features_test_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_test_H1 = features_test_H1.map(lambda x: list(itertools.chain(*x)))

        # Step 8. Convert all features of H0 and H1 into a DataFrame format, and merge them.
        
        features_train_H0 = pd_series_to_dataframe(features_train_H0)
        
        features_train_H1 = pd_series_to_dataframe(features_train_H1)

        features_test_H0 = pd_series_to_dataframe(features_test_H0)
        
        features_test_H1 = pd_series_to_dataframe(features_test_H1)

        features_train = pd.concat([features_train_H0, features_train_H1], axis=1)   
        
        features_train.columns = np.arange(features_train.shape[1])
        
        features_test = pd.concat([features_test_H0, features_test_H1], axis=1)
        
        features_test.columns = np.arange(features_test.shape[1])

        # Step 9. Convert all features in each row into a list format.
        
        train_features = []
        
        features_train = features_train.values.tolist()
        
        for row in features_train:
            
            train_features.append(row)

        test_features = []
        
        features_test = features_test.values.tolist()
        
        for row in features_test:
            
            test_features.append(row)

        # Step 10. Combine all features into the original DataFrame.
        
        train_data['features'] = train_features
        
        test_data['features'] = test_features

        # Step 11. Machine Learning
        
        train_features = np.vstack(train_data['features'].values)
        
        train_target = train_data['target'].values
        
        test_features = np.vstack(test_data['features'].values)
        
        test_target = test_data['target'].values

        bc_accuracy, bc_macro_f1 = strawberry_classification_model(train_features, train_target, test_features, test_target)

        # Step 12. Organize the classification results into a DataFrame.
        
        classification_accuracy = [bc_accuracy]

        classification_macor_f1 = [bc_macro_f1]
        
        all_classification_accuracy.append(classification_accuracy)

        all_classification_macro_f1.append(classification_macor_f1)
        
        print('window size ', size,' execution completed !')

        print()

    all_classification_accuracy = np.array(all_classification_accuracy)                                      
    
    all_classification_accuracy = pd.DataFrame(all_classification_accuracy, columns = ['Accuracy'])   
    
    all_classification_accuracy.index = window_size

    all_classification_macro_f1 = np.array(all_classification_macro_f1)                                      
    
    all_classification_macro_f1 = pd.DataFrame(all_classification_macro_f1, columns = ['Macro-F1'])   
    
    all_classification_macro_f1.index = window_size
 
    return all_classification_accuracy, all_classification_macro_f1


In [33]:
# wine main experiment

def wine_experiment(embedding_dimension = 2):

    all_classification_accuracy = []   # Store all accuracy results.
    all_classification_macro_f1 = []   # Store all macro-f1 results.
    
    # Step 1. Call the wine_data() method to obtain the beef dataset and convert it into Series format.
    
    train_data, test_data = wine_data()

    train_data_series = pd.Series(train_data['signal'].iloc[:])
    
    test_data_series = pd.Series(test_data['signal'].iloc[:])

    window_size = [5, 10, 45, 50, 95, 100]        # (5, 10) = small size，(45, 50) = medium size，(95, 100) = large size

    # Step 2. Apply a sliding window method to all time series in the dataset.
    
    for size in window_size:
        
        train_sliding_windows = train_data_series.map(lambda x: sliding_windows(x, size))
        
        test_sliding_windows = test_data_series.map(lambda x: sliding_windows(x, size))

        # Step 3. Apply time-delay embedding method to each window and obtain a 2-dimensional point cloud.
        
        train_point_cloud = train_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])
        
        test_point_cloud = test_sliding_windows.map(lambda x: [takens_embedding(row, embedding_dimension) for row in x])

        # Step 4. Convert each point cloud into a persistence diagram.
        
        train_persistence_diagrams = train_point_cloud.map(lambda x: [persistent_homology(row) for row in x])
        
        test_persistence_diagrams = test_point_cloud.map(lambda x: [persistent_homology(row) for row in x])

        # Step 5. Separate the H0 and H1 structures in the persistence diagram.
        
        train_H0 = train_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        train_H1 = train_persistence_diagrams.map(lambda x: [row[1] for row in x])

        test_H0 = test_persistence_diagrams.map(lambda x: [row[0] for row in x])
        
        test_H1 = test_persistence_diagrams.map(lambda x: [row[1] for row in x])

        # Step 6. Feature Extraction !

        features_train_H0 = train_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_train_H1 = train_H1.map(lambda x: [get_diagram_features(row) for row in x])

        features_test_H0 = test_H0.map(lambda x: [get_diagram_features(row) for row in x])
        
        features_test_H1 = test_H1.map(lambda x: [get_diagram_features(row) for row in x])

        # Step 7. Use the itertools package to merge the features of each window into a 1-dimensional array.
        
        features_train_H0 = features_train_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_train_H1 = features_train_H1.map(lambda x: list(itertools.chain(*x)))

        features_test_H0 = features_test_H0.map(lambda x: list(itertools.chain(*x)))
        
        features_test_H1 = features_test_H1.map(lambda x: list(itertools.chain(*x)))

        # Step 8. Convert all features of H0 and H1 into a DataFrame format, and merge them.
        
        features_train_H0 = pd_series_to_dataframe(features_train_H0)
        
        features_train_H1 = pd_series_to_dataframe(features_train_H1)

        features_test_H0 = pd_series_to_dataframe(features_test_H0)
        
        features_test_H1 = pd_series_to_dataframe(features_test_H1)

        features_train = pd.concat([features_train_H0, features_train_H1], axis=1)   
        
        features_train.columns = np.arange(features_train.shape[1])
        
        features_test = pd.concat([features_test_H0, features_test_H1], axis=1)
        
        features_test.columns = np.arange(features_test.shape[1])

        # Step 9. Convert all features in each row into a list format.
        
        train_features = []
        
        features_train = features_train.values.tolist()
        
        for row in features_train:
            
            train_features.append(row)

        test_features = []
        
        features_test = features_test.values.tolist()
        
        for row in features_test:
            
            test_features.append(row)

        # Step 10. Combine all features into the original DataFrame.
        
        train_data['features'] = train_features
        
        test_data['features'] = test_features

        # Step 11. Machine Learning
        
        train_features = np.vstack(train_data['features'].values)
        
        train_target = train_data['target'].values
        
        test_features = np.vstack(test_data['features'].values)
        
        test_target = test_data['target'].values

        bc_accuracy, bc_macro_f1 = wine_classification_model(train_features, train_target, test_features, test_target)

        # Step 12. Organize the classification results into a DataFrame.
        
        classification_accuracy = [bc_accuracy]

        classification_macor_f1 = [bc_macro_f1]
        
        all_classification_accuracy.append(classification_accuracy)

        all_classification_macro_f1.append(classification_macor_f1)
        
        print('window size ', size,' execution completed !')

        print()

    all_classification_accuracy = np.array(all_classification_accuracy)                                      
    
    all_classification_accuracy = pd.DataFrame(all_classification_accuracy, columns = ['Accuracy'])   
    
    all_classification_accuracy.index = window_size

    all_classification_macro_f1 = np.array(all_classification_macro_f1)                                      
    
    all_classification_macro_f1 = pd.DataFrame(all_classification_macro_f1, columns = ['Macro-F1'])   
    
    all_classification_macro_f1.index = window_size
 
    return all_classification_accuracy, all_classification_macro_f1


## （7）Experimental results

### <font color = green >The classification Accuracy and Macro-F1 score of the Beef dataset</font>

In [36]:
# beef

beef_accuracy, beef_macro_f1 = beef_experiment(embedding_dimension = 2)

beef_accuracy

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters: {'bc__n_estimators': 50, 'bc__random_state': 14}
Best cross-validation accuracy: 0.5999999999999999
bc Test accuracy: 0.9333333333333333
bc Test Macro-F1: 0.9328671328671329
window size  5  execution completed !

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters: {'bc__n_estimators': 50, 'bc__random_state': 14}
Best cross-validation accuracy: 0.6666666666666667
bc Test accuracy: 0.8666666666666667
bc Test Macro-F1: 0.8664935064935065
window size  10  execution completed !

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters: {'bc__n_estimators': 50, 'bc__random_state': 14}
Best cross-validation accuracy: 0.5333333333333333
bc Test accuracy: 0.7333333333333333
bc Test Macro-F1: 0.7321212121212122
window size  45  execution completed !

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters: {'bc__n_estimators': 50, 'bc__random_state': 14}
Best c

,Accuracy
5,93.333
10,86.667
45,73.333
50,70.000
95,53.333
100,50.000


In [37]:
beef_macro_f1

,Macro-F1
5,93.287
10,86.649
45,73.212
50,71.524
95,53.212
100,49.758


In [38]:
with open('Beef_Research_result.csv',  'w', encoding = 'utf = 8') as file:
    
     beef_accuracy.to_csv(file)

with open('Beef_MacroF1_result.csv',  'w', encoding = 'utf = 8') as file:
    
     beef_macro_f1.to_csv(file)

### <font color = green >The classification Accuracy and Macro-F1 score of the Coffee dataset</font>

In [39]:
# coffee

coffee_accuracy, coffee_macro_f1 = coffee_experiment(embedding_dimension = 2)

coffee_accuracy

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best parameters: {'bc__max_samples': 1.0, 'bc__n_estimators': 100, 'bc__random_state': 14}
Best cross-validation accuracy: 0.9629629629629629
bc Test accuracy: 0.8928571428571429
bc Test Macro-F1: 0.8893280632411067
window size  5  execution completed !

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best parameters: {'bc__max_samples': 1.0, 'bc__n_estimators': 100, 'bc__random_state': 14}
Best cross-validation accuracy: 1.0
bc Test accuracy: 1.0
bc Test Macro-F1: 1.0
window size  10  execution completed !

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best parameters: {'bc__max_samples': 1.0, 'bc__n_estimators': 100, 'bc__random_state': 14}
Best cross-validation accuracy: 0.9629629629629629
bc Test accuracy: 0.9285714285714286
bc Test Macro-F1: 0.9282051282051282
window size  45  execution completed !

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best parameters: {'bc__max_samples': 1

,Accuracy
5,89.286
10,100.000
45,92.857
50,92.857
95,100.000
100,100.000


In [40]:
coffee_macro_f1

,Macro-F1
5,88.933
10,100.000
45,92.821
50,92.821
95,100.000
100,100.000


In [41]:
with open('Coffee_Research_result.csv',  'w', encoding = 'utf = 8') as file:
    
     coffee_accuracy.to_csv(file)

with open('Coffee_MacroF1_result.csv',  'w', encoding = 'utf = 8') as file:
    
     coffee_macro_f1.to_csv(file)

### <font color = green >The classification Accuracy and Macro-F1 score of the Ham dataset</font>

In [42]:
# ham

ham_accuracy, ham_macro_f1 = ham_experiment(embedding_dimension = 2)

ham_accuracy

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters: {'bc__max_features': 0.8, 'bc__max_samples': 0.8, 'bc__n_estimators': 10, 'bc__random_state': 45}
Best cross-validation accuracy: 0.5878378378378378
bc Test accuracy: 0.6952380952380952
bc Test Macro-F1: 0.695210449927431
window size  5  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters: {'bc__max_features': 0.8, 'bc__max_samples': 1.0, 'bc__n_estimators': 50, 'bc__random_state': 45}
Best cross-validation accuracy: 0.6331331331331331
bc Test accuracy: 0.7333333333333333
bc Test Macro-F1: 0.7327272727272728
window size  10  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters: {'bc__max_features': 0.8, 'bc__max_samples': 1.0, 'bc__n_estimators': 20, 'bc__random_state': 45}
Best cross-validation accuracy: 0.6053553553553553
bc Test accuracy: 0.8
bc Test Macro-F1: 0.7993447993447994
window size  45  execution complet

,Accuracy
5,69.524
10,73.333
45,80.000
50,74.286
95,64.762
100,66.667


In [43]:
ham_macro_f1

,Macro-F1
5,69.521
10,73.273
45,79.934
50,74.248
95,64.749
100,66.667


In [44]:
with open('Ham_Research_result.csv',  'w', encoding = 'utf = 8') as file:
    
     ham_accuracy.to_csv(file)

with open('Ham_MacroF1_result.csv',  'w', encoding = 'utf = 8') as file:
    
     ham_macro_f1.to_csv(file)

### <font color = green >The classification Accuracy and Macro-F1 score of the Meat dataset</font>

In [45]:
# meat

meat_accuracy, meat_macro_f1 = meat_experiment(embedding_dimension = 2)

meat_accuracy

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 1.0, 'bc__n_estimators': 10, 'bc__random_state': 42}
Best cross-validation accuracy: 0.9833333333333334
bc Test accuracy: 0.95
bc Test Macro-F1: 0.9499358151476253
window size  5  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 1.0, 'bc__n_estimators': 10, 'bc__random_state': 42}
Best cross-validation accuracy: 1.0
bc Test accuracy: 0.95
bc Test Macro-F1: 0.9499358151476253
window size  10  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 0.8, 'bc__n_estimators': 10, 'bc__random_state': 42}
Best cross-validation accuracy: 1.0
bc Test accuracy: 0.95
bc Test Macro-F1: 0.9499389499389498
window size  45  execution completed !

Fitting 3 folds for each of 30 candidates, totalli

,Accuracy
5,95.000
10,95.000
45,95.000
50,95.000
95,98.333
100,100.000


In [46]:
meat_macro_f1

,Macro-F1
5,94.994
10,94.994
45,94.994
50,94.994
95,98.332
100,100.000


In [47]:
with open('Meat_Research_result.csv',  'w', encoding = 'utf = 8') as file:
    
     meat_accuracy.to_csv(file)

with open('Meat_MacroF1_result.csv',  'w', encoding = 'utf = 8') as file:
    
     meat_macro_f1.to_csv(file)

### <font color = green >The classification Accuracy and Macro-F1 score of the Olive oil dataset</font>

In [48]:
# oliveoil

oliveoil_accuracy, oliveoil_macro_f1 = oliveoil_experiment(embedding_dimension = 2)

oliveoil_accuracy

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 1.0, 'bc__n_estimators': 50, 'bc__random_state': 14}
Best cross-validation accuracy: 0.7666666666666666
bc Test accuracy: 0.9
bc Test Macro-F1: 0.9065079365079365
window size  5  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max

Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 0.6, 'bc__n_estimators': 10, 'bc__random_state': 14}
Best cross-validation accuracy: 0.8333333333333334
bc Test accuracy: 0.9666666666666667
bc Test Macro-F1: 0.9759725400457666
window size  10  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 1.0, 'bc__n_estimators': 100, 'bc__random_state': 14}
Best cross-validation accuracy: 0.9
bc Test accuracy: 0.9333333333333333
bc Test Macro-F1: 0.9513888888888888
window size  45  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 0.6, 'bc__n_estimators': 30, 'bc__random_state': 14}
Best cross-validation accuracy: 0.9
bc Test accuracy: 0.9666666666666667
bc Test Macro-F1: 0.9759725400457666
window size  50  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters: {'bc__max_features': 0.8, 'bc__max_samples': 1.0, 'bc__n_estimators': 10, 'bc__random_state': 14}
Best cross-validation accuracy: 0.9
bc Test accuracy: 0.9666666666666667
bc Test Macro-F1: 0.9759725400457666
window size  95  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 1.0, 'bc__n_estimators': 10, 'bc__random_state': 14}
Best cross-validation accuracy: 0.8666666666666667
bc Test accuracy: 0.9666666666666667
bc Test Macro-F1: 0.9759725400457666
window size  100  execution completed !



,Accuracy
5,90.000
10,96.667
45,93.333
50,96.667
95,96.667
100,96.667


In [49]:
oliveoil_macro_f1

,Macro-F1
5,90.651
10,97.597
45,95.139
50,97.597
95,97.597
100,97.597


In [50]:
with open('Oliveoil_Research_result.csv',  'w', encoding = 'utf = 8') as file:
    
     oliveoil_accuracy.to_csv(file)

with open('Oliveoil_MacroF1_result.csv',  'w', encoding = 'utf = 8') as file:
    
     oliveoil_macro_f1.to_csv(file)

### <font color = green >The classification Accuracy and Macro-F1 score of the Strawberry dataset</font>

In [51]:
# strawberry

strawberry_accuracy, strawberry_macro_f1 = strawberry_experiment(embedding_dimension = 2)

strawberry_accuracy

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/opt/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 1.0, 'bc__n_estimators': 20, 'bc__random_state': 14}
Best cross-validation accuracy: 0.9509963334927467
bc Test accuracy: 0.9783783783783784
bc Test Macro-F1: 0.9764451235039471
window size  5  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/opt/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 1.0, 'bc__n_estimators': 20, 'bc__random_state': 14}
Best cross-validation accuracy: 0.9412083532600032
bc Test accuracy: 0.9567567567567568
bc Test Macro-F1: 0.9532001011889704
window size  10  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/opt/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 0.8, 'bc__n_estimators': 30, 'bc__random_state': 14}
Best cross-validation accuracy: 0.9248923959827833
bc Test accuracy: 0.9432432432432433
bc Test Macro-F1: 0.9390545214956351
window size  45  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/opt/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 0.6, 'bc__n_estimators': 20, 'bc__random_state': 14}
Best cross-validation accuracy: 0.9200063765343535
bc Test accuracy: 0.927027027027027
bc Test Macro-F1: 0.9218768328445748
window size  50  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 1.0, 'bc__n_estimators': 50, 'bc__random_state': 14}
Best cross-validation accuracy: 0.8955364259524948
bc Test accuracy: 0.9108108108108108
bc Test Macro-F1: 0.9060848697417911
window size  95  execution completed !

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters: {'bc__max_features': 1.0, 'bc__max_samples': 0.8, 'bc__n_estimators': 50, 'bc__random_state': 14}
Best cross-validation accuracy: 0.8971544715447154
bc Test accuracy: 0.9081081081081082
bc Test Macro-F1: 0.9036015325670498
window size  100  execution completed !



,Accuracy
5,97.838
10,95.676
45,94.324
50,92.703
95,91.081
100,90.811


In [52]:
strawberry_macro_f1

,Macro-F1
5,97.645
10,95.320
45,93.905
50,92.188
95,90.608
100,90.360


In [53]:
with open('Strawberry_Research_result.csv',  'w', encoding = 'utf = 8') as file:
    
     strawberry_accuracy.to_csv(file)

with open('Strawberry_MacroF1_result.csv',  'w', encoding = 'utf = 8') as file:
    
     strawberry_macro_f1.to_csv(file)

### <font color = green >The classification Accuracy and Macro-F1 score of the Wine dataset</font>

In [54]:
# wine

wine_accuracy, wine_macro_f1 = wine_experiment(embedding_dimension = 2)

wine_accuracy

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best parameters: {'bc__max_samples': 0.6, 'bc__n_estimators': 30, 'bc__random_state': 3}
Best cross-validation accuracy: 0.7017543859649122
bc Test accuracy: 0.8333333333333334
bc Test Macro-F1: 0.8328173374613004
window size  5  execution completed !

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best parameters: {'bc__max_samples': 1.0, 'bc__n_estimators': 30, 'bc__random_state': 3}
Best cross-validation accuracy: 0.7894736842105264
bc Test accuracy: 0.9259259259259259
bc Test Macro-F1: 0.9258241758241759
window size  10  execution completed !

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best parameters: {'bc__max_samples': 0.8, 'bc__n_estimators': 10, 'bc__random_state': 3}
Best cross-validation accuracy: 0.6140350877192983
bc Test accuracy: 0.6481481481481481
bc Test Macro-F1: 0.6480274442538594
window size  45  execution completed !

Fitting 3 folds for each of 12 candidates, totalling 36 

,Accuracy
5,83.333
10,92.593
45,64.815
50,70.370
95,83.333
100,79.630


In [55]:
wine_macro_f1

,Macro-F1
5,83.282
10,92.582
45,64.803
50,70.330
95,83.282
100,79.623


In [56]:
with open('Wine_Research_result.csv',  'w', encoding = 'utf = 8') as file:
    
     wine_accuracy.to_csv(file)

with open('Wine_MacroF1_result.csv',  'w', encoding = 'utf = 8') as file:
    
     wine_macro_f1.to_csv(file)